# analisis

importamos Dataset de kaggle.

In [1]:
# Sample Python code for dataset download
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhbagchi/books-dataset")

print("Path to dataset files:", path)

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 24.6M/24.6M [00:06<00:00, 3.75MB/s]

Extracting files...


Path to dataset files: /home/vscode/.cache/kagglehub/datasets/saurabhbagchi/books-dataset/versions/1


In [ ]:
import pandas as pd
import os

dataset_path = "/home/vscode/.cache/kagglehub/datasets/saurabhbagchi/books-dataset/versions/1"

# List files in the dataset directory
files = os.listdir(dataset_path)
print("Files in dataset directory:", files)

Files in dataset directory: ['books_data']


Buscamos los csv de interes: 
1. books.csv
2. ratings.csv
3. users.csv

In [ ]:
import csv

# Define the dataset path
dataset_path = "/home/vscode/.cache/kagglehub/datasets/saurabhbagchi/books-dataset/versions/1/books_data"

# List files inside books_data
files = os.listdir(dataset_path)
print("Files in books_data directory:", files)

# Filter for CSV or JSON files
data_files = [f for f in files if f.endswith(('.csv', '.json'))]
print("Data files found:", data_files)

# Load and display data
if data_files:
    for file_name in data_files:
        data_path = os.path.join(dataset_path, file_name)  # Iterate over all files
        print(f"\nLoading file: {file_name}")
        
        try:
            # Load CSV with proper delimiter and quoting
            if data_path.endswith('.csv'):
                try:
                    df = pd.read_csv(
                        data_path, 
                        encoding='utf-8', 
                        sep=';',                # Use semicolon as delimiter
                        quotechar='"',          # Handle quoted fields
                        on_bad_lines='skip',    # Skip bad lines
                        engine='python'         # More robust parsing
                    )
                except (UnicodeDecodeError, pd.errors.ParserError):
                    print("UTF-8 decoding failed or parsing issue, trying ISO-8859-1 encoding...")
                    df = pd.read_csv(
                        data_path, 
                        encoding='ISO-8859-1', 
                        sep=';', 
                        quotechar='"',
                        on_bad_lines='skip', 
                        engine='python'
                    )

            # Load JSON
            elif data_path.endswith('.json'):
                df = pd.read_json(data_path)
            
            # Display dataset details
            print(f"\nFirst 5 rows of {file_name}:")
            print(df.head())

            print(f"\n{file_name} Info:")
            print(df.info())

            print(f"\nDescriptive Statistics for {file_name}:")
            print(df.describe())

        except Exception as e:
            print(f"Error loading {file_name}: {e}")

else:
    print("No CSV or JSON data files found in books_data.")

Files in books_data directory: ['books.csv', 'users.csv', 'ratings.csv']
Data files found: ['books.csv', 'users.csv', 'ratings.csv']

Loading file: books.csv
UTF-8 decoding failed or parsing issue, trying ISO-8859-1 encoding...

First 5 rows of books.csv:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author  Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                 2002     Oxford University Press   
1  Richard Bruce Wright                 2001       HarperFlamingo Canada   
2          Carlo D'Este                 1991             HarperPerennial   
3      Gina Bari Kolata        

Creación de `dataframes`

In [22]:
dataset_path = "/home/vscode/.cache/kagglehub/datasets/saurabhbagchi/books-dataset/versions/1/books_data"

books_path = os.path.join(dataset_path, "books.csv")
users_path = os.path.join(dataset_path, "users.csv")
ratings_path = os.path.join(dataset_path, "ratings.csv")

books_df = pd.read_csv(books_path, sep=';', quotechar='"', encoding='ISO-8859-1', on_bad_lines='skip', engine='python')
users_df = pd.read_csv(users_path, sep=';', quotechar='"', encoding='ISO-8859-1', on_bad_lines='skip', engine='python')
ratings_df = pd.read_csv(ratings_path, sep=';', quotechar='"', encoding='ISO-8859-1', on_bad_lines='skip', engine='python')

## Exploracion del dataset

In [14]:
# Explore missing values and data types

def explore_dataframe(df, name):
    print(f"\n🗂️ Exploring {name} dataset:")
    print("-" * 40)
    print(f"Shape: {df.shape}")
    print("\nColumns and Data Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nUnique Values per Column:")
    print(df.nunique())
    print("-" * 40)

# Explore each dataset
explore_dataframe(books_df, "Books")
explore_dataframe(users_df, "Users")
explore_dataframe(ratings_df, "Ratings")


🗂️ Exploring Books dataset:
----------------------------------------
Shape: (270491, 8)

Columns and Data Types:
ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication     int64
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object

Missing Values:
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

Unique Values per Column:
ISBN                   270491
Book-Title             241282
Book-Author            101696
Year-Of-Publication       116
Publisher               16727
Image-URL-S            270175
Image-URL-M            270175
Image-URL-L            270175
dtype: int64
----------------------------------------

🗂️ Exploring Users dataset:
----------------------------------------
Shape: (278

In [ ]:
ratings_df["Book-Rating"].describe()

count    1.149766e+06
mean     2.866906e+00
std      3.854172e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.000000e+00
max      1.000000e+01
Name: Book-Rating, dtype: float64